In [22]:
import numpy as np
import pandas as pd
import scipy.stats as stats

def generate_patient_vitals(id, n, t=3):
    np.random.seed(id)
    
    # Simulate random values for heart rate (bpm)
    heart_rate = np.round(stats.truncnorm.rvs(-t, t, 80, 12, n), 2)
    
    # Simulate random values for blood pressure (mmHg)
    systolic_bp = np.round(stats.truncnorm.rvs(-t, t, 105, 12, n), 2)
    diastolic_bp = np.round(stats.truncnorm.rvs(-t, t, 70, 10, n), 2)

    # Simulate random values for temperature (°C)
    temperature = np.round(stats.truncnorm.rvs(-t, t, 37, 0.5, n), 2)

    # Simulate random values for respiration rate (breaths per minute)
    respiration_rate = np.round(stats.truncnorm.rvs(-t, t, 17, 3, n), 2)

    # Simulate random values for SpO2 (%)
    spo2 = np.round(stats.truncnorm.rvs(-t, t, 98, 1.5, n), 2)
    spo2[spo2 > 100] = 100

    return pd.DataFrame({
        'Patient ID': id,
        'Heart Rate': heart_rate,
        'Systolic BP': systolic_bp,
        'Diastolic BP': diastolic_bp,
        'Temperature': temperature,
        'Respiration Rate': respiration_rate,
        'SpO2': spo2
    })

In [23]:
import json

# Processing address dataset
geojson_file_path = '../ma_addresses/city_of_boston-addresses-city.geojson'

# Extract relevant information
addresses = []
with open(geojson_file_path, 'r') as file:
    for line in file:
        try:
            data = json.loads(line)
            properties = data.get("properties", {})
            number = properties.get("number", "")
            street = properties.get("street", "")
            postcode = properties.get("postcode", "")

            # Create consolidated address
            consolidated_address = f"{number} {street}, {postcode}"

            # Append to addresses list
            addresses.append(consolidated_address)

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

# Create a DataFrame
address_df = pd.DataFrame({"Address": addresses})

# Checking Addresses
address_df.head()

,Address
0,"6-10 A Street, 02136"
1,"7 A Street, 02136"
2,"10 A Street, 02127"
3,"172-174 A Street, 02210"
4,"176-178 A Street, 02210"


In [24]:
Patient_count = 10

In [25]:
patient_df = pd.DataFrame([])
for i in range(1, Patient_count+1):
    patient_df = pd.concat([patient_df, generate_patient_vitals(i, 100)])
address_df_sample = address_df.sample(n = Patient_count).reset_index().drop('index', axis=1, errors='ignore')    

In [26]:
address_df_sample

,Address
0,"68 W Concord Street, 02118"
1,"142 Norwell Street, 02121"
2,"1666 Commonwealth Avenue, 02135"
3,"52 Columbia Road, 02121"
4,"456 Centre Street, 02130"
5,"38 Greenough Avenue, 02130"
6,"1086-1088A Dorchester Avenue, 02125"
7,"30 Elm Street, 02130"
8,"73 Maple Street, 02136"
9,"69-69A Readville Street, 02136"


In [36]:
from faker import Faker
import random

fake = Faker()

def generate_random_phone_number():
    # Generate a random US phone number for Boston area code (617)
    area_code = "617"
    middle_digits = f"{random.randint(200, 999):03d}"
    last_digits = f"{random.randint(1000, 9999):04d}"
    return f"({area_code}) {middle_digits}-{last_digits}"

def generate_random_ward_number():
    # Generate a random ward number (assuming a range)
    return random.randint(1, 20)

def generate_random_age():
    return random.randint(18, 75)

# Generate random patients with names, phone numbers, and ward numbers
names = []
phnumber = []
ward_list = []
age_list = []
for _ in range(Patient_count):
    names.append(fake.name())
    phnumber.append(generate_random_phone_number())
    ward_list.append(generate_random_ward_number())
    age_list.append(generate_random_age())

patient_id_list = list(range(1, Patient_count+1))
    
detail_df = pd.DataFrame({
    "Patient ID": patient_id_list,
    "Patient Name": names, 
    "Phone Number": phnumber,
    "Age": age_list,
    "Ward Number": ward_list
    })
detail_df

,Patient ID,Patient Name,Phone Number,Age,Ward Number
0,1,Jerry Obrien,(617) 408-8730,75,8
1,2,Jill Davis,(617) 300-8864,53,18
2,3,Shannon Diaz,(617) 823-2893,53,14
3,4,Adam Acevedo,(617) 289-2588,44,4
4,5,Diane Hanson,(617) 964-2997,68,3
5,6,Todd Martin,(617) 849-5404,44,19
6,7,Mr. Jeff Adams,(617) 732-7672,46,2
7,8,Rachel Harris,(617) 343-3655,62,5
8,9,Bradley Davis,(617) 988-7246,29,2
9,10,Lauren Brewer,(617) 837-2428,28,4


In [37]:
# Joining detail DF and addresses DF
patient_info_df = detail_df.join(address_df_sample)
patient_info_df

,Patient ID,Patient Name,Phone Number,Age,Ward Number,Address
0,1,Jerry Obrien,(617) 408-8730,75,8,"68 W Concord Street, 02118"
1,2,Jill Davis,(617) 300-8864,53,18,"142 Norwell Street, 02121"
2,3,Shannon Diaz,(617) 823-2893,53,14,"1666 Commonwealth Avenue, 02135"
3,4,Adam Acevedo,(617) 289-2588,44,4,"52 Columbia Road, 02121"
4,5,Diane Hanson,(617) 964-2997,68,3,"456 Centre Street, 02130"
5,6,Todd Martin,(617) 849-5404,44,19,"38 Greenough Avenue, 02130"
6,7,Mr. Jeff Adams,(617) 732-7672,46,2,"1086-1088A Dorchester Avenue, 02125"
7,8,Rachel Harris,(617) 343-3655,62,5,"30 Elm Street, 02130"
8,9,Bradley Davis,(617) 988-7246,29,2,"73 Maple Street, 02136"
9,10,Lauren Brewer,(617) 837-2428,28,4,"69-69A Readville Street, 02136"


In [38]:
patient_df.columns

Index(['Patient ID', 'Heart Rate', 'Systolic BP', 'Diastolic BP',
       'Temperature', 'Respiration Rate', 'SpO2'],
      dtype='object')

In [39]:
patient_df.describe()

,Patient ID,Heart Rate,Systolic BP,Diastolic BP,Temperature,Respiration Rate,SpO2
count,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000
mean,5.500000,79.757730,104.64872,70.003520,37.010400,17.047100,97.868930
std,2.873719,11.320872,12.03627,9.678028,0.496171,2.943674,1.441492
min,1.000000,44.300000,70.63000,40.770000,35.650000,8.490000,93.520000
25%,3.000000,72.595000,96.19500,63.525000,36.690000,15.140000,96.850000
50%,5.500000,79.860000,105.09500,70.115000,37.015000,17.075000,97.930000
75%,8.000000,86.985000,113.11250,76.900000,37.350000,19.045000,99.012500
max,10.000000,109.410000,136.80000,96.250000,38.430000,25.300000,100.000000


In [41]:
patient_df.to_csv("patient_vitals_CSV/patient_vitals.csv")
patient_info_df.to_csv("patient_vitals_CSV/patient_info.csv", index=False)